In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Calculate Distances") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
import string
dfMetroAreasRaw = spark.read.load("../rawdata/Gaz_ua_national.txt", format="csv", delimiter="\t", header=True, inferSchema=True)
dfZCTAsRaw = spark.read.load("../rawdata/Gaz_zcta_national.txt", format="csv", delimiter="\t", header=True, inferSchema=True)
dfMetroAreasRaw = dfMetroAreasRaw.withColumnRenamed(dfMetroAreasRaw.columns[-1],dfMetroAreasRaw.columns[-1].strip(string.whitespace))
dfZCTAsRaw = dfZCTAsRaw.withColumnRenamed(dfZCTAsRaw.columns[-1],dfZCTAsRaw.columns[-1].strip(string.whitespace))
dfMetroAreasRaw.printSchema()
dfZCTAsRaw.printSchema()

root
 |-- GEOID: integer (nullable = true)
 |-- NAME: string (nullable = true)
 |-- UATYPE: string (nullable = true)
 |-- POP10: integer (nullable = true)
 |-- HU10: integer (nullable = true)
 |-- ALAND: long (nullable = true)
 |-- AWATER: integer (nullable = true)
 |-- ALAND_SQMI: double (nullable = true)
 |-- AWATER_SQMI: double (nullable = true)
 |-- INTPTLAT: double (nullable = true)
 |-- INTPTLONG: double (nullable = true)

root
 |-- GEOID: integer (nullable = true)
 |-- POP10: integer (nullable = true)
 |-- HU10: integer (nullable = true)
 |-- ALAND: long (nullable = true)
 |-- AWATER: long (nullable = true)
 |-- ALAND_SQMI: double (nullable = true)
 |-- AWATER_SQMI: double (nullable = true)
 |-- INTPTLAT: double (nullable = true)
 |-- INTPTLONG: double (nullable = true)



In [71]:
import numpy as np
def deg2rad(deg):
    return deg/360*(2*np.pi)



In [88]:
dfMetroAreas = dfMetroAreasRaw.select('GEOID','NAME','UATYPE','POP10','HU10','ALAND_SQMI',\
                                      'AWATER_SQMI','INTPTLAT','INTPTLONG')\
                            .withColumnRenamed('GEOID','m_id')\
                            .withColumnRenamed('NAME', 'name')\
                            .withColumnRenamed('POP10','m_pop')\
                            .withColumnRenamed('HU10','m_house_unit')\
                            .withColumnRenamed('ALAND_SQMI','m_land')\
                            .withColumnRenamed('AWATER_SQMI','m_water')\
                            .withColumnRenamed('INTPTLAT', 'm_lat_d')\
                            .withColumnRenamed('INTPTLONG', 'm_long_d')

dfMetroAreas = dfMetroAreas.withColumn('m_lat_r', deg2rad(dfMetroAreas.m_lat_d)).withColumn('m_long_r', deg2rad(dfMetroAreas.m_long_d))
print(dfMetroAreas.count())
dfMetroAreas.show()

3592
+----+--------------------+------+------+------------+-------+-------+---------+-----------+-------------------+-------------------+
|m_id|                name|UATYPE| m_pop|m_house_unit| m_land|m_water|  m_lat_d|   m_long_d|            m_lat_r|           m_long_r|
+----+--------------------+------+------+------------+-------+-------+---------+-----------+-------------------+-------------------+
|  37|Abbeville, LA Urb...|     C| 19824|        8460| 11.283|  0.116|29.967602| -92.098219| 0.5230333238272378|-1.6074171567727997|
|  64|Abbeville, SC Urb...|     C|  5243|        2578|  4.369|  0.008|34.179237| -82.379726| 0.5965402214694692|-1.4377974555907782|
|  91|Abbotsford, WI Ur...|     C|  3966|        1616|  2.071|  0.005|44.948612| -90.315875| 0.7845012736014334|-1.5763093855696337|
| 118|Aberdeen, MS Urba...|     C|  4666|        2050|  2.864|   0.02|33.824742| -88.554591| 0.5903531165376119|-1.5455691807069378|
| 145|Aberdeen, SD Urba...|     C| 25977|       12114| 12.742|  

In [89]:
dfZCTAs = dfZCTAsRaw.select('GEOID','POP10','HU10','ALAND_SQMI','AWATER_SQMI','INTPTLAT','INTPTLONG')\
                .withColumnRenamed('GEOID','z_id')\
                .withColumnRenamed('POP10','z_pop')\
                .withColumnRenamed('HU10','z_house_unit')\
                .withColumnRenamed('ALAND_SQMI','z_land')\
                .withColumnRenamed('AWATER_SQMI','z_water')\
                .withColumnRenamed('INTPTLAT', 'z_lat_d')\
                .withColumnRenamed('INTPTLONG', 'z_long_d')


dfZCTAs = dfZCTAs.withColumn('z_lat_r', deg2rad(dfZCTAs.z_lat_d)).withColumn('z_long_r', deg2rad(dfZCTAs.z_long_d))
print(dfZCTAs.count())
dfZCTAs.show()

33120
+----+-----+------------+------+-------+---------+----------+-------------------+-------------------+
|z_id|z_pop|z_house_unit|z_land|z_water|  z_lat_d|  z_long_d|            z_lat_r|           z_long_r|
+----+-----+------------+------+-------+---------+----------+-------------------+-------------------+
| 601|18570|        7744|64.348|  0.309|18.180555|-66.749961|0.31731054458991764|-1.1650065950278068|
| 602|41520|       18073|30.613|  1.717|18.362268| -67.17613| 0.3204820347335941|-1.1724446472477383|
| 603|54689|       25653|31.614|  0.071|18.455183|-67.119887| 0.3221037074080847|-1.1714630217165392|
| 606| 6615|        2877|42.309|  0.005|18.158345|-66.932911|0.31692290696304976|-1.1681996748943304|
| 610|29016|       12618|35.916|  1.611|18.290955|-67.125868| 0.3192373880841194| -1.171567409859101|
| 612|67010|       30992|67.609|  3.787|18.402239|  -66.7114| 0.3211796602889088|-1.1643335786149451|
| 616|11017|        4896|11.533|  0.058|18.420412|-66.671979| 0.321496838973

In [90]:
dfDist = dfZCTAs.join(dfMetroAreas)
print(dfZCTAs.count(),dfMetroAreas.count(),dfZCTAs.count()*dfMetroAreas.count(),dfDist.count())
dfDist.printSchema()
dfDist.show()


33120 3592 118967040 118967040
root
 |-- z_id: integer (nullable = true)
 |-- z_pop: integer (nullable = true)
 |-- z_house_unit: integer (nullable = true)
 |-- z_land: double (nullable = true)
 |-- z_water: double (nullable = true)
 |-- z_lat_d: double (nullable = true)
 |-- z_long_d: double (nullable = true)
 |-- z_lat_r: double (nullable = true)
 |-- z_long_r: double (nullable = true)
 |-- m_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- UATYPE: string (nullable = true)
 |-- m_pop: integer (nullable = true)
 |-- m_house_unit: integer (nullable = true)
 |-- m_land: double (nullable = true)
 |-- m_water: double (nullable = true)
 |-- m_lat_d: double (nullable = true)
 |-- m_long_d: double (nullable = true)
 |-- m_lat_r: double (nullable = true)
 |-- m_long_r: double (nullable = true)

+----+-----+------------+------+-------+---------+----------+-------------------+-------------------+----+--------------------+------+------+------------+-------+-------+---------

In [91]:
# Wikipedia, "Great-circle distance," https://en.wikipedia.org/wiki/Great-circle_distance retrieved 12/9/2016
from pyspark.sql.functions import acos, cos, sin, abs

RMETERS = 6371000 #meters
RMILES = RMETERS*0.000621371

dfDist = dfDist.withColumn('dist',acos(
                            sin(dfDist.z_lat_r)*sin(dfDist.m_lat_r)+
                            cos(dfDist.z_lat_r)*cos(dfDist.m_lat_r)*cos(abs(dfDist.z_long_r-dfDist.m_long_r))
                            )*RMILES)



In [ ]:
dfDist.write.save("../processeddata/distances.parquet",mode='overwrite')